In [2]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt

# add directory with ribasim-repository
sys.path.append("..\\..\\ribasim\\python\\ribasim")
import ribasim

# add directory with ribasim_lumping_nl-repository
sys.path.append("..")
import ribasim_lumping_nl

import pandas as pd
import geopandas as gpd
import numpy as np
import pyarrow as pa

from numba.core.errors import NumbaDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)
pd.options.mode.chained_assignment = None

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
start_time = pd.Timestamp.now()

#### BASIS OPZETTEN

In [5]:
# define network name, base dir
network_name = "HunzeenAas"

base_dir = Path("..\\..\\LHM\\")

# directory hydamo data
hydamo_basis_dir = Path(base_dir, "01_data_bewerking_analyse\\3_verwerkte_data", network_name)
# directory results
results_dir = Path(base_dir, "02_ribasim_netwerk_schematisatie\\", network_name)
# link to ribasim_executable
path_ribasim_executable = "..\\..\\..\\ribasim_cli\\bin\\ribasim.exe"

In [6]:
# # define network name, base dir
# network_name = "tki_hooge_raam"
# base_dir = Path("..\\..\\ribasim_lumping_data\\")
# # directory D-HYDRO basis calculations
# dhydro_basis_dir = Path(base_dir, network_name, "2_dhydro\\basis\\")
# # directory D-HYDRO results calculations
# dhydro_results_dir = Path(base_dir, network_name, "2_dhydro\\results\\")
# # directory results
# results_dir = Path(base_dir, network_name, "3_ribasim\\")
# # batch file used to calculate volume D-Hydro module per GridPoint
# dhydro_volume_tool_bat_file = Path(r"c:\Program Files\Deltares\D-HYDRO Suite 2023.03 1D2D\plugins\DeltaShell.Dimr\kernels\x64\dflowfm\scripts\run_dfm_volume_tool.bat")
# dhydro_volume_tool_increment = 0.1
# # link to ribasim_executable
# path_ribasim_executable = "..\\..\\..\\ribasim_cli\\bin\\ribasim.exe"

In [7]:
hydamo_basis_dir

WindowsPath('../../LHM/01_data_bewerking_analyse/3_verwerkte_data/HunzeenAas')

In [33]:
# Create networkanalysis
network = ribasim_lumping_nl.create_ribasim_lumping_network(
    base_dir=base_dir,
    name=network_name,
    # dhydro_basis_dir=dhydro_basis_dir,
    # dhydro_results_dir=dhydro_results_dir,
    hydamo_basis_dir=hydamo_basis_dir,
    results_dir=results_dir,
    path_ribasim_executable=path_ribasim_executable,
    crs=28992
)
# network.export_to_geopackage()

In [9]:
network

RibasimLumpingNetwork(name='HunzeenAas', base_dir=WindowsPath('../../LHM'), hydamo_basis_dir=WindowsPath('../../LHM/01_data_bewerking_analyse/3_verwerkte_data/HunzeenAas'), results_dir=WindowsPath('../../LHM/02_ribasim_netwerk_schematisatie/HunzeenAas'), path_ribasim_executable=WindowsPath('../../../ribasim_cli/bin/ribasim.exe'), areas_gdf=None, his_data=None, map_data=None, network_data=None, volume_data=None, network_graph=None, branches_gdf=None, network_nodes_gdf=None, edges_gdf=None, nodes_gdf=None, structures_gdf=None, stations_gdf=None, pumps_gdf=None, weirs_gdf=None, orifices_gdf=None, bridges_gdf=None, culverts_gdf=None, uniweirs_gdf=None, boundaries_gdf=None, laterals_gdf=None, boundaries_data=None, laterals_data=None, simulation_code=None, basin_areas_gdf=None, basins_gdf=None, split_nodes=None, basin_connections_gdf=None, boundary_connections_gdf=None, split_node_type_conversion=None, split_node_id_conversion=None, nodes_h_df=None, nodes_a_df=None, nodes_v_df=None, basins_h

#### INLEZEN HYDAMO  DATA

In [34]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.add_basis_network(
    source_type='hydamo'
);

Define locations where the network should be split into Ribasim basins:

In [35]:
# black list: "105BGA"
network.add_split_nodes(
    weirs=True,
    pumps=False,
    uniweirs=False,
    edges=False,
    # structures_ids_to_include=[],
    # structures_ids_to_exclude=[],
    # edge_ids_to_include=[],                                  
    # edge_ids_to_exclude=[],
);

2153 split locations
 - stuw: 2153


In [36]:
network.split_nodes

,split_node,node_no,edge_no,split_node_id,geometry,object_type,split_type,code
0,1,-1,NaN,NaN,POINT (265070.246 558275.993),stuw,NaN,KST-W-11023
1,2,-1,NaN,NaN,POINT (257084.871 591778.878),stuw,NaN,KST-D-14950
2,3,-1,NaN,NaN,POINT (270818.725 560262.502),stuw,NaN,KST-W-11264
3,4,-1,NaN,NaN,POINT (234960.327 555456.881),stuw,NaN,KST-A-01053
4,5,-1,NaN,NaN,POINT (236221.874 554836.147),stuw,NaN,KST-A-01089
...,...,...,...,...,...,...,...,...
2148,2149,-1,NaN,NaN,POINT (240296.030 576768.233),stuw,NaN,KIN-H-10014
2149,2150,-1,NaN,NaN,POINT (256370.612 558101.919),stuw,NaN,KST-H-10087
2150,2151,-1,NaN,NaN,POINT (241036.477 583953.318),stuw,NaN,KIN-D-00014
2151,2152,-1,NaN,NaN,POINT (255455.795 582950.428),stuw,NaN,KST-O-17750


In [ ]:
network.split_nodes.split_node_id.values

Specify translation split_nodes to ribasim-nodes per type or for specifiek split_node_id

In [ ]:
split_node_type_conversion = dict(
    weir="TabulatedRatingCurve",
    uniweir="TabulatedRatingCurve",
    pump="Pump",
    culvert="TabulatedRatingCurve",
    edge="ManningResistance",
    orifice="Outlet",
    boundary_connection="ManningResistance"
)
split_node_id_conversion = {}

Create basins (gdf) based on nodes, edges, split_node_ids and areas

In [ ]:
network.generate_ribasim_lumping_network(
    simulation_code=simulation_code,
    split_node_type_conversion=split_node_type_conversion,
    split_node_id_conversion=split_node_id_conversion,
);

Generate Ribasim model and export to geopackage

In [ ]:
ribasim_model = network.generate_ribasim_model_complete(
    set_name=set_name,
    starttime=starttime,
    endtime=endtime
)

In [ ]:
# network.plot_basin_waterlevels_for_basins(
#     set_name='winter', 
#     basins_nos=[6, 9, 10, 39, 19, 57, 58, 53, 20, 38]
# );

In [ ]:
f"Duration: {pd.Timestamp.now() - start_time}"

In [ ]:
basin_df, control_df, flow_df = network.read_ribasim_results(simulation_code=simulation_code)
basin_df.drop(columns='node_id').groupby('time').sum()['storage'].plot()

In [ ]:
flow_df[~flow_df['edge_id'].isna()].groupby('to_node_id').get_group(1)['flow'].plot()

In [ ]:
# q = uniweirs_q_gdf.loc['weirgen_discharge'].iloc[5]['winter']

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.optimize import curve_fit

# for i, weir in uniweirs_q_gdf.loc['weirgen_discharge']['winter'].iterrows():
#     x_data = weir.index.values
#     y_data = weir.values

#     def exponential_function(x, a, b):
#         return a * np.exp(b * x)

#     params, covariance = curve_fit(exponential_function, x_data, y_data)
#     a, b = params

#     y_fit = exponential_function(x_data, a, b)

#     fig, ax = plt.subplots()
#     ax.scatter(x_data, y_data, label="Data")
#     ax.plot(x_data, y_fit, "-x", label="Exponential Fit", color='red')
#     fig.show()
#     # plt.legend()
#     # plt.xlabel('X')
#     # plt.ylabel('Y')
#     # plt.title('Exponential Regression')
#     # plt.show()

In [ ]:
# def exponential_function(x, a, b):
#     return a * np.exp(b * x)

# def apply_exponential_function(df_series):
#     x_data = df_series.index.values
#     y_data = df_series.values
#     params, covariance = curve_fit(exponential_function, x_data, y_data)
#     a, b = params
#     y_fit = exponential_function(x_data, a, b)
#     df_series.values = y_fit
#     return df_series

# uniweirs_q_gdf#.loc['winter']#.apply(lambda row: apply_exponential_function(row))